In [1]:
!pip install ctgan
!pip install sdv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from ctgan import CTGAN 

import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

dataset = pd.read_csv('https://raw.githubusercontent.com/mahayasa/various-sampling-churn-prediction/main/data/telco-non-internet.csv')

dataset['state']=le.fit_transform(dataset['state'])
dataset['area_code']=le.fit_transform(dataset['area_code'])
dataset['international_plan']=le.fit_transform(dataset['international_plan'])
dataset['voice_mail_plan']=le.fit_transform(dataset['voice_mail_plan'])
dataset['churn']=le.fit_transform(dataset['churn'])

df1=dataset[dataset['churn']==0]
df=dataset[dataset['churn']==1]


batch_size = 5000
epochs = 100
model = CTGAN(batch_size=batch_size, epochs=epochs, verbose=True)
model.fit(df)



Epoch 1, Loss G:  0.0649,Loss D: -0.0304
Epoch 2, Loss G:  0.0568,Loss D: -0.0447
Epoch 3, Loss G:  0.0574,Loss D: -0.0672
Epoch 4, Loss G:  0.0552,Loss D: -0.0723
Epoch 5, Loss G:  0.0423,Loss D: -0.0890
Epoch 6, Loss G:  0.0223,Loss D: -0.1023
Epoch 7, Loss G: -0.0120,Loss D: -0.1093
Epoch 8, Loss G: -0.0541,Loss D: -0.1189
Epoch 9, Loss G: -0.1071,Loss D: -0.1285
Epoch 10, Loss G: -0.1842,Loss D: -0.0887
Epoch 11, Loss G: -0.2757,Loss D: -0.0633
Epoch 12, Loss G: -0.3723,Loss D: -0.0391
Epoch 13, Loss G: -0.4766,Loss D: -0.0210
Epoch 14, Loss G: -0.5472,Loss D:  0.0140
Epoch 15, Loss G: -0.6225,Loss D:  0.0431
Epoch 16, Loss G: -0.7212,Loss D:  0.0521
Epoch 17, Loss G: -0.8324,Loss D:  0.0373
Epoch 18, Loss G: -0.8795,Loss D:  0.0478
Epoch 19, Loss G: -0.9313,Loss D:  0.0400
Epoch 20, Loss G: -0.9536,Loss D:  0.0453
Epoch 21, Loss G: -0.9557,Loss D:  0.0040
Epoch 22, Loss G: -0.9618,Loss D:  0.0223
Epoch 23, Loss G: -0.9937,Loss D: -0.0170
Epoch 24, Loss G: -0.9546,Loss D: -0.1311
E

In [3]:
n_generated_data = 3054
generated_df = model.sample(n_generated_data)

In [4]:
#concat original data and gan data
data_concat = pd.concat([df, generated_df])
# combine data churn and not churn
data=pd.concat([df1, data_concat])
data

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,35,107,1,0,1,26,161.600000,123,27.470000,195.500000,103,16.620000,254.400000,103,11.450000,13.700000,3,3.700000,1,0
1,31,137,1,0,0,0,243.400000,114,41.380000,121.200000,110,10.300000,162.600000,104,7.320000,12.200000,5,3.290000,0,0
2,35,84,0,1,0,0,299.400000,71,50.900000,61.900000,88,5.260000,196.900000,89,8.860000,6.600000,7,1.780000,2,0
3,36,75,1,1,0,0,166.700000,113,28.340000,148.300000,122,12.610000,186.900000,121,8.410000,10.100000,3,2.730000,3,0
4,19,121,2,0,1,24,218.200000,88,37.090000,348.500000,108,29.620000,212.600000,118,9.570000,7.500000,7,2.030000,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3049,-3,180,0,0,0,0,172.425034,95,56.734183,337.544335,124,8.784065,210.934161,93,6.623654,9.197414,9,4.643398,1,1
3050,19,203,1,0,0,0,235.422942,107,22.981235,156.735681,98,17.928464,75.274122,142,10.341949,12.537010,7,4.229063,3,1
3051,35,155,0,0,0,0,186.994006,82,37.087642,231.591593,183,2.872718,209.202118,122,8.783319,12.051782,4,2.588744,1,1
3052,11,173,0,1,0,24,209.971748,55,50.191892,127.399560,113,7.038587,194.444300,87,7.657591,7.092392,7,3.890489,5,1


In [9]:
# Do Classification Proccess
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix ,accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics as mt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import VotingClassifier
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import NeighbourhoodCleaningRule
from imblearn.under_sampling import TomekLinks

data=data.dropna()
data=data.fillna(0)

X=data.drop(['churn'],axis=1)
y=data["churn"]

#enn = EditedNearestNeighbours(n_neighbors=3)
#X, y = enn.fit_resample(X, y)
ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
X, y = ncr.fit_resample(X, y)

#tomek_links = TomekLinks()
#X, y = tomek_links.fit_resample(X, y)

#kfold cross validation
cv = KFold(n_splits=5, random_state=1, shuffle=True)
# get a list of base models
def get_models():
	models = list()
	models.append(('xb', XGBClassifier(colsample_bytree=0.6, max_depth = 5, gamma=1.5, min_child_weight=5)))
	models.append(('rf', RandomForestClassifier(n_estimators =100, criterion = 'gini', random_state = 0)))
#	models.append(('ab', AdaBoostClassifier(n_estimators=100,learning_rate=0.1,algorithm='SAMME.R')))
	return models
 
# evaluate each base model
def evaluate_models(models, X,y):
	# fit and evaluate the models
	scores = list()
	for name, model in models:
		# evaluate the model
		acc=cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
		# store the performance
		scores.append(mean(acc))
		# report model performance
	return scores




# create the base models
models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X,y)
print(scores)
# create the ensemble
ensemble = VotingClassifier(estimators=models, voting='soft', weights=scores)
score=cross_val_score(ensemble, X, y, scoring='f1', cv=cv, n_jobs=-1)
print('F1 score: %.3f' % (mean(score)))
print('STD F1 Score: %.3f' % (std(score)))

rc=cross_val_score(ensemble, X, y, scoring='recall', cv=cv, n_jobs=-1)
print('Recall: %.3f' % (mean(rc)))

from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score

# Define the specificity scorer
scorer = make_scorer(specificity_score)

# Calculate the cross-validated specificity score
sp = cross_val_score(ensemble, X, y, cv=5, scoring=scorer)
print('Specitifity: %.3f' % (mean(sp)))

auc=cross_val_score(ensemble, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('AUC ROC: %.3f' % (mean(auc)))

import math

gmean=rc*sp
gmean=mean(gmean)
sqrtg = math.sqrt(gmean)
print('G-Mean: %.3f' % (sqrtg))

[0.9980459601415118, 0.9974974765656984]
F1 score: 0.980
STD F1 Score: 0.007
Recall: 0.975
Specitifity: 0.989
AUC ROC: 0.998
G-Mean: 0.982
